LanghChain Q&A 아키텍쳐    
크게 질문 처리와 답변 생성 두 가지 주요 단계로 구성   

질문 처리   
- 데이터 소스 연결 : 질문에 대한 답변을 찾기 위해 다양한 텍스트 데이터 소스에 연결해야함
- 데이터 인덱싱 및 검색 : 데이터 소스에서 관련 정보를 효율적으로 찾기 위해, 데이터는 인덱싱 되어야함.   
   
답변 생성
- 고급 자연어 처리 모델(NLP)을 사용하여 검색 된 데이터로부터 답변을 생성할 수 있는 능 제공


**아키텍쳐**   
인덱싱 & 검색 및 생성   

   
인덱싱   
1. LOAD : 데이터를 로드 (DocumentLoaders 사용) 
2. SPLIT : Documents를더 작은 청크로 나눔
3. EMBED , STORE : 나중에 검색할 수 있도록 분할을 저장
   
검색 및 생성   
1. 검색 : 사용자 입력이 주어지면 Retriever를 사용하여 저장소에서 검색
2. 생성 : ChatModel / LLM은 질문과 검색된 데이터를 포함한 프롬프트를 사용하여 답변 생생



In [4]:
# API KEY 설정정
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import bs4     # 웹 페이지 크롤링 용용
from langchain import hub  #hub에서 프롬프트 끌어오기
from langchain.text_splitter import RecursiveCharacterTextSplitter #텍스트 분할
from langchain_community.document_loaders import WebBaseLoader #문서 로딩
from langchain_community.vectorstores import FAISS  #벡터 저장
from langchain_core.output_parsers import StrOutputParser #출력 파싱
from langchain_core.runnables import RunnablePassthrough #실행 가능한 패스스루
from langchain_openai import ChatOpenAI, OpenAIEmbeddings #OpenAI의 챗봇, 임베딩 기능

In [8]:
bs4.SoupStrainer(
    "div",
    attrs={"class" : ["newsct_article _article_body", "media_end_head_title"]},
)

<SoupStrainer name=[<TagNameMatchRule string=div pattern=None function=None present=None>] attrs=defaultdict(<class 'list'>, {'class': [<AttributeValueMatchRule string=newsct_article _article_body pattern=None function=None present=None>, <AttributeValueMatchRule string=media_end_head_title pattern=None function=None present=None>]}) string=[]>

In [14]:
#네이버 기사
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000434525?sid=102",),
    #web_paths=("https://www.hankookilbo.com/News/Read/A2025032410280003768",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body",
                             "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000434525?sid=102'}, page_content="\n안동까지 덮친 화마…의성 대형 산불, '강풍' 타고 확산\n\n\n소방청, 경북 안동에 국가 소방동원령 추가 발령\n\n\n\n\n\n\n\n\n\n\n\n\n\n[앵커]오늘(25일)까지도 전국 곳곳에서 대형 산불이 계속되고 있습니다. 경남 산청은 닷새째, 경북 의성은 나흘째입니다. 특히, 의성에선 산불이 강한 바람을 타고 안동까지 번지는 중입니다. 취재기자 연결해 자세히 알아보겠습니다.이재승 기자, 안동에도 국가소방동원령이 발령됐다고요?[기자]그렇습니다, 지금 산불 상황이 가장 심각한 곳이 경북 의성입니다.강한 바람을 타고 산불이 인근 지역으로 퍼져나가면서 의성과 안동 사이 고속도로 휴게소에 불이 붙기도 했습니다.소방청은 어젯밤 10시 45분에 경북 안동에 국가 소방동원령을 추가 발령했습니다.앞서 지난 22일 토요일엔 경남 산청과 울산 울주, 경북 의성에 국가 소방동원령이 내려졌습니다.국가 소방동원령 발령에 따라 전국 소방차 총 320대가 이번 동시다발 산불 발생 지역에 동원됐습니다.[앵커]며칠째 산불이 이어지는데, 피해 규모도 계속 커지고 있죠?[기자]지금 산불은 마치 '도깨비불'처럼 번지는 중입니다.지금 불이 난 곳을 진화해도 또 예상치 못한 다른 곳으로 확산하고 있다는 얘기인데요.강풍 때문에 산불의 작은 불티가 순식간에 멀리까지 날아가 또 다른 산불을 만드는 겁니다.피해가 눈덩이처럼 불어나면서, 정부는 경남 산청에 이어 울산 울주군과 경북 의성군, 경남 하동군도 특별재난지역으로 선포했습니다.산불이 닷새째 이어지면서 피해가 예상되는 면적인 산불영향구역이 전국적으로 총 1만㏊를 넘어섰습니다.경북 의성 산불 피해면적이 8천㏊로 가장 넓은데 진화율은 아직도 60% 수준에 머물고 있습니다.이번 산불로 인한 인명 피해는 사망 4명에 중상 5명, 경상 6명 등 총 15명입니다.또

In [39]:
#한국일보 기사
loader2 = WebBaseLoader(
    web_paths=("https://www.hankookilbo.com/News/Read/A2025032410280003768",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": "col-main"},
        )
    ),
)
docs2 = loader2.load()
print(f"문서의 수: {len(docs2)}")
docs2





문서의 수: 1


[Document(metadata={'source': 'https://www.hankookilbo.com/News/Read/A2025032410280003768', 'language': 'No language found.'}, page_content='\n재판관 미임명·의결 정족수 등쟁점별로 재판관 판단 엇갈려尹·국힘 추천 재판관 2인 \'각하\'\n\n\n\n\n\n이미지 확대보기\n\n\n헌법재판소가 한덕수 대통령 권한대행 국무총리에 대한 국회의 탄핵소추를 기각한 24일 한 총리가 정부서울청사로 업무에 복귀하며 총리실로 향하고 있다. 심현철 기자\n\n\n내란 방조 등 이유로 탄핵심판대에 올랐던 한덕수 대통령 권한대행 국무총리 탄핵 사건이 24일 기각됐다. \'12·3 불법계엄\'으로 국회에서 탄핵소추된 사건 중 처음으로 나온 결론이다. 헌법재판소는 윤석열 대통령의 비상계엄 선포와 관련해선 별도로 판단하지 않았다.\n헌재는 이날 재판관 8명 중 5인의 기각 의견과 1인의 인용 의견, 2인의 각하 의견으로 한 총리 탄핵 사건을 기각했다. 탄핵이 인용되려면 재판관 6인 이상의 찬성이 필요하다. 헌재 결정으로 한 총리는 탄핵소추안이 의결된 지 87일, 변론이 종결된 지 33일 만에 직무에 복귀했다.\n한 총리는 지난해 12월 27일 △윤 대통령의 12·3 불법계엄 공모·묵인·방조 △국회 추천 헌법재판관 후보자 임명 거부 △내란 상설특별검사(특검) 임명 절차 이행 회피 △김건희 특검법 등 거부 △여당과 \'공동 국정 운영\' 시도로 헌법·법률 위배 등을 이유로 탄핵소추됐다.\n\n\n\n\n재판관들 의견은 크게 기각 5인, 인용 1인, 각하 2인으로 갈렸다. 기각 의견을 낸 5인 중에서도 쟁점에 따라 의견이 달랐다. 문형배·이미선·김형두·정정미 재판관은 재판관 후보자 미임명 외 다른 사유들에 대해 "사실을 인정할 만한 증거나 객관적 자료를 찾을 수 없다"는 이유로 법 위반을 인정하지 않았다. 국회가 선출한 재판관 후보자 3인을 임명하지 않은 것에 대해선 "위법하지

In [40]:
from bs4 import BeautifulSoup

editor_texts = []
for doc in docs2:
    soup = BeautifulSoup(doc.page_content, "html.parser")
    ps = soup.find_all("p", class_=["editor-p", "read"])
    for p in ps:
        editor_texts.append(p.get_text(strip=True))

# 결과 확인
for t in editor_texts:
    print(t)

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits= text_splitter.split_documents(docs)


# 앞서 로딩된 웹페이지의 text를 RecursiveCharacterTextSplitter 로 청크로 나눔눔

In [16]:
#FAISS는 청크를 이용하여 벡터화 하는 애임. OPenaIEmbedding의 임베딩 기능을 활용하여 앞의 청크를 벡터 저장소에 벡터 방식으로 저장
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

#뉴스 기사의 정보를 검색할 검색기
retriever = vectorstore.as_retriever()

In [17]:
#rlm이 소유한 rag-prompt를 pull한것것

prompt = hub.pull("rlm/rag-prompt")
prompt

c:\Users\GoodusData\Desktop\공부\langchain\1lecture\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [18]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#체인 생성
rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
rag_chain.invoke(
    "이번 산불의 가장 큰 피해지역은 어디인가요?"
)  

'경북 의성이 이번 산불의 가장 큰 피해지역입니다. 산불이 강한 바람을 타고 안동까지 번지고 있습니다. 경북 의성 산불 피해면적이 8천㏊로 가장 넓습니다.'

In [21]:
rag_chain.invoke(
    "인명 피해 정도는 어느정도죠?"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

'인명 피해 정도는 사망 4명에 중상 5명, 경상 6명 등 총 15명이며, 주택과 창고, 사찰, 공장 등 건물 130여개가 불에 탔습니다. 산불로 인한 피해가 예상되는 면적인 산불영향구역이 전국적으로 총 1만㏊를 넘어섰습니다. 경북 의성 산불 피해면적이 8천㏊로 가장 넓은데 진화율은 아직도 60% 수준에 머물고 있습니다.'

In [22]:
rag_chain.invoke(
    "비가 오면 좋을텐데, 강수 확률은 어느정도일까요??"
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

'내일 모레 목요일에 전국에 단비가 예상되며, 수도권에는 5에서 20㎜의 강우량이 예상됩니다. 산불이 집중된 영남내륙에는 5㎜ 안팎의 비가 예상되며, 예상 강우량은 많지 않지만 산불을 제압하는데 도움이 될 것으로 예상됩니다. 현재 산불로 인한 피해가 심각하며, 산불이 계속 확산되고 있습니다.'

In [ ]:
#시스템의 저장 공간 확보 및 불필요한 데이터의 혼란을 방지하기 위한 컬렉션 삭제
vectorstore.delete_collection()